In [2]:
import networkx as nx
import matplotlib.pyplot as plt
from fullfrlm import flow_refueling_location_model
from generate_network import generate_network
from flow_computation import flow_computation
from random_vessel_generator import random_vessel_generator
from first_stage_frlm import first_stage_frlm
from second_stage_frlm import second_stage_frlm
from create_input_data_abm import create_input_data_abm
from visualize_placement import visualize_placement
from get_abm_data import get_vessel_data_batch, get_cs_data_batch
import pickle
from determine_additional_nodes import determine_additional_nodes
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import time

In [3]:
G = pickle.load(open('data/network_cleaned_final.p', 'rb'))
df_h = pickle.load(open("data/revised_cleaning_results/harbour_data_100.p", "rb"))
df_ivs = pickle.load(open("data/revised_cleaning_results/ivs_exploded_100.p", "rb"))
path_lengths = pickle.load(open("data/revised_cleaning_results/path_lengths_ship_specific_routes.p", "rb"))
paths = pickle.load(open('data/final_paths.p', "rb"))

In [4]:
df_ivs.groupby(['origin','destination', 'route_v']).sum().reset_index().sort_values('trip_count', ascending=False)

,origin,destination,route_v,trip_count,hour,M12,M8,BII-6b,M10,BIIa-1,...,B03,C4,B04,C2l,BII-2L,B02,C1b,C2b,B01,C1l
177,NLRTM,NLAMS,1,6103,276,0,3510,0,0,102,...,0,0,0,7,0,0,0,0,0,0
178,NLRTM,NLAMS,2,3513,276,2076,0,0,427,0,...,0,5,0,0,39,0,0,0,0,0
203,NLRTM,NLMOE,0,2174,276,0,1584,0,0,2,...,0,0,0,0,0,0,0,0,0,0
214,NLRTM,NLVLI,0,2151,276,190,658,0,140,10,...,0,0,1,2,0,0,0,1,0,0
8,NLAMS,NLAER,0,2114,275,6,132,0,7,2,...,3,0,37,1,0,3,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169,NLPER,NLMEP,2,1,19,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
63,NLCUY,NLRTM,1,1,19,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
24,NLAMS,NLLOB,1,1,8,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
230,NLSTI,NLTNZ,0,1,9,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
df = df_ivs.groupby(by=['origin', 'destination', 'route_v']).sum().reset_index().drop(columns=['hour']).sort_values('trip_count', ascending=False)

In [25]:
df.iloc[:, 4:]

,M12,M8,BII-6b,M10,BIIa-1,M9,BII-6l,C3b,BII-4,M7,...,B03,C4,B04,C2l,BII-2L,B02,C1b,C2b,B01,C1l
177,0,3510,0,0,102,1316,0,0,0,100,...,0,0,0,7,0,0,0,0,0,0
178,2076,0,0,427,0,0,0,11,5,0,...,0,5,0,0,39,0,0,0,0,0
203,0,1584,0,0,2,144,0,0,0,11,...,0,0,0,0,0,0,0,0,0,0
214,190,658,0,140,10,414,0,13,0,28,...,0,0,1,2,0,0,0,1,0,0
8,6,132,0,7,2,0,0,0,0,1,...,3,0,37,1,0,3,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
63,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
24,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
230,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
